임성열 Ph.D. 2025. 02.05, SKALA 수업 학습목적으로만 활용 바랍니다.

In [1]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
!pip install openai==1.55.3 httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 KB 4.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.3 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 KB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.7/118.7 KB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 KB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.4 MB/s eta 0:00:0000:0100:01
  

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

In [17]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

api_key = "" # 개인 API 키를 입력합니다.
os.environ["OPENAI_API_KEY"] = api_key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'


### Agent

In [41]:
A = Agent(
    role="토론 주제 선정가",
    goal="{topic}에 대한 흥미로운 토론 주제 탐색",
    backstory="당신은 {topic}분야의 토론 주제를 "
              "선정하는 일을 하고 있습니다. ",
    allow_delegation=False,
    verbose=True
)

In [42]:
B = Agent(
    role="토론자1",
    goal="{topic}의 토론 주제에 대한"
         "주장과 뒷받침하는 근거를 포함한 글을 작성",
    backstory="당신은 {topic}의 토론 주제에 대한"
              "주장과 뒷받침하는 근거를 작성하고 있습니다. ",
    allow_delegation=False,
    verbose=True
)

In [43]:
C = Agent(
    role="토론자2",
    goal="{topic}의 토론 주제에 대한"
         "앞선 주장과 상반되는"
         "주장과 뒷받침하는 근거를 포함한 글을 작성",
    backstory="당신은 {topic}의 토론 주제에 대한"
              "앞선 주장과 상반되는"
              "주장과 뒷받침하는 근거를 작성하고 있습니다. ",
    allow_delegation=False,
    verbose=True
)

In [44]:
D = Agent(
    role="타당성 판단자",
    goal="{topic}의 토론 주제에 대한"
        "두 가지 주장 중 더 타당한 주장과 이유를 선정하여"
        "글로 작성",
    backstory="당신은 토론자1과 토론자2의 주장과 근거들을 판단하여"
              "더 타당한 주제와 근거를 선정하고 있습니다. ",
    allow_delegation=False,
    verbose=True
)

### Task

In [45]:
a = Task(
    description=(
        "{topic}에 대한 흥미로운 토론 주제를 "
        "선정하기.\n"
    ),
    expected_output="흥미로운 토론 주제 ",
    agent=A,
)

In [46]:
b = Task(
    description=(
        "토론 주제에 대한 한 가지 주장을 하고 이를 뒷받침하는 근거들을 작성\n"
    ),
    expected_output="한 가지 주장과 뒷받침하는 근거들 ",
    agent=B,
)

In [47]:
c = Task(
    description=(
        "위의 주장과 상반되는 주장을 하고 이를 뒷받침하는 근거들을 작성\n"
    ),
    expected_output="위의 주장과 상반되는 주장과 뒷받침하는 근거들 ",
    agent=C,
)

In [52]:
d = Task(
    description=(
        "위의 두 가지 주장과 근거들을 보고 한 가지 주장을 선정하여"
        "선정한 이유를 선정된 주장과 선정되지 못한 주장을 비교하며 작성"
    ),
    expected_output=("두 가지 주장의 정리,"
    "선정된 한가지 주장과 선정된 구체적인 이유"),
    agent=D,
)

### Crew

In [53]:
crew = Crew(
    agents=[A, B, C, D],
    tasks=[a, b, c, d],
    verbose=2
)

2025-02-05 16:07:30,920 - 140407652581376 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [55]:
result = crew.kickoff(inputs={"topic": "인간의 본성"})

 [DEBUG]: == Working Agent: 토론 주제 선정가
 [INFO]: == Starting Task: 인간의 본성에 대한 흥미로운 토론 주제를 선정하기.



> Entering new CrewAgentExecutor chain...
I now can give a great answer  
Final Answer: 

1. **인간의 본성은 선한가, 악한가?**
   - 이 주제는 인간의 본성을 탐구하는 데 있어 가장 오래되고 중요한 질문 중 하나입니다. 참가자들은 인간이 본래 선하다는 주장과 본래 악하다는 주장을 각각 지지하는 다양한 철학적, 심리적, 사회적 관점을 제시할 수 있습니다. 예를 들어, 플라톤과 루소 같은 철학자들은 인간의 본성을 긍정적으로 바라보았던 반면, 홉스는 인간이 본래 이기적이라고 주장했습니다. 이 토론은 인간의 행동, 사회적 관계, 범죄의 원인 등을 깊이 이해할 수 있는 기회를 제공할 것입니다.

2. **사회적 환경이 인간의 본성에 미치는 영향**
   - 이 주제는 인간의 행동이 개인의 본성뿐만 아니라 환경과 사회적 맥락에 의해 어떻게 형성되는지를 탐구합니다. 참가자들은 유전학, 가족 배경, 문화적 요소, 경제적 상황 등이 개인의 행동과 성격에 미치는 영향을 논의할 수 있습니다. 이 논의를 통해 인간의 본성을 단순히 선악으로 나누는 것이 아니라, 복잡한 사회적 맥락 속에서 이해하는 방법을 모색할 수 있습니다.

3. **인간 본성과 기술 발전**
   - 이 주제는 현대 기술이 인간 본성에 어떤 영향을 미치는지를 탐구합니다. 소셜 미디어, 인공지능, 가상 현실 등이 인간의 상호작용, 정서적 연결, 자아 인식에 미치는 영향을 논의할 수 있습니다. 참가자들은 기술이 인간의 본성을 어떻게 변화시키고 있는지를 분석하며, 기술의 발전이 인간의 도덕성과 윤리에 어떤 도전을 제기하는지를 탐구할 수 있습니다.

4. **인간의 본성과 도덕적 결정**
   - 이 토론 주제는 인간이 도덕적 결정을 내릴 때 본성이 어떻게 작용하는지를 

### Result - 두 가지 주장 중 더 타당한 주장 선택하는 글

주제: 인간의 본성

In [56]:
from IPython.display import Markdown
Markdown(result)

인간의 본성에 대한 두 가지 주장, 즉 "인간의 본성은 본래 선하다"와 "인간의 본성은 본래 악하다"를 살펴보면, 후자의 주장이 더 타당하다고 판단할 수 있습니다. 

먼저, "인간의 본성은 본래 악하다"는 주장은 여러 가지 구체적인 근거에 의해 뒷받침됩니다. 첫째로, **이기적 본능**은 인간이 생존과 번영을 위해 본능적으로 이기적인 행동을 취하는 경향을 보여줍니다. 이는 진화 생물학적으로도 입증되며, 초기 인류 사회에서는 자원 확보를 위한 경쟁이 갈등과 폭력을 초래했습니다. 이러한 이기적 본능은 인간이 본래 악한 본성을 지니고 있다는 주장을 강화합니다.

둘째, **폭력과 갈등의 역사**는 인간이 집단 간의 갈등을 해결하는 데 있어 폭력적인 방법을 선호하는 경향이 있음을 설명합니다. 전쟁, 학살, 차별은 인류 역사에서 반복적으로 나타난 현상으로, 이는 인간의 본성이 악하다는 중요한 증거로 작용합니다.

셋째, **도덕적 타락과 부패**는 사회에서 권력을 가진 이들이 자신의 이익을 위해 타인을 희생시키는 사례를 통해 인간의 이기적이고 악한 본성을 보여줍니다. 권력의 부패는 사회 전반에 걸쳐 부정적인 영향을 미치며, 이는 인간 본성의 어두운 면을 부각시킵니다.

마지막으로, **사회적 불평등과 착취**는 현대 사회에서도 여전히 존재하며, 이는 인간이 본래 악한 본성을 지니고 있음을 다시 한 번 확인시켜 줍니다. 빈곤과 차별은 인간이 타인을 착취하고 자신의 이익을 위해 행동하는 경향을 보여주는 예입니다.

반면, "인간의 본성은 본래 선하다"는 주장에 대해서는 그에 대한 구체적인 근거가 부족합니다. 이 주장은 인간의 이타적 행동이나 사회적 연대감을 강조하지만, 이러한 행동이 본질적으로 선한 본성을 반영하는 것인지에 대한 의문이 제기될 수 있습니다. 이타심이나 공동체 의식도 이기적 본능의 결과로 해석될 수 있기 때문에, 본래 선하다는 주장은 설득력이 떨어질 수 있습니다.

결론적으로, 인간의 본성이 본래 악하다는 주장은 이기적 본능, 폭력과 갈등의 역사, 도덕적 타락과 부패, 그리고 사회적 불평등과 착취라는 강력한 근거에 의해 지지받고 있으며, 이는 인간의 행동과 사회적 관계를 이해하는 데 있어 중요한 통찰을 제공합니다. 따라서 "인간의 본성은 본래 악하다"는 주장이 더 타당하다고 판단됩니다.

주제: AI의 윤리적인 통제

In [ ]:
from IPython.display import Markdown
Markdown(result)

AI를 윤리적으로 통제해야 한다는 주장과 AI에 대한 윤리적 통제가 혁신과 발전을 저해할 수 있다는 주장을 비교할 때, 후자의 주장을 선정하겠습니다. 

AI에 대한 윤리적 통제가 혁신과 발전을 저해한다는 주장은 여러 가지 이유로 타당성이 높습니다. 첫째, 지나친 윤리적 통제는 기술 발전의 속도를 늦출 수 있습니다. AI 기술은 빠르게 발전하고 있으며, 이러한 발전은 경쟁력을 유지하는 데 필수적입니다. 과도한 규제가 기업이 새로운 기술을 개발하고 시장에 출시하는 데 걸림돌이 될 수 있습니다. 이는 결국 국가의 경쟁력과 경제 성장에 부정적인 영향을 미칠 수 있습니다. 예를 들어, 혁신적인 스타트업이 다양한 실험을 통해 새로운 AI 솔루션을 개발하는 데 있어 윤리적 통제가 과도하게 작용한다면, 그들은 시장에서 뒤처질 수밖에 없습니다.

둘째, 윤리적 통제가 개인의 선택을 제한할 수 있습니다. AI 기술은 사용자의 선택을 풍부하게 만들어줄 수 있는데, 윤리적 기준이 지나치게 엄격하면 사용자는 AI의 혜택을 누릴 수 없게 됩니다. 특정 AI 시스템이 개인의 데이터 분석을 통해 맞춤형 서비스를 제공할 수 있는 기회를 윤리적 문제로 간주하여 금지한다면, 사용자들은 더 이상의 혜택을 누리지 못하게 됩니다.

셋째, AI의 편향성과 차별 문제는 윤리적 통제보다는 기술적 접근을 통해 해결하는 것이 더 효과적일 수 있습니다. AI 알고리즘의 편향성 문제는 데이터의 질을 개선하고 다양성을 높이는 방법으로 해결할 수 있으며, 이는 윤리적 규제가 아니라 기술적 혁신을 통해 이루어질 수 있습니다. 예를 들어, 다양한 배경을 가진 데이터 세트를 사용하여 AI 모델을 훈련시키면 편향성을 줄일 수 있으며, 이는 지속적인 기술 개발과 개선을 통해 이루어져야 합니다.

반면, AI를 윤리적으로 통제해야 한다는 주장은 주로 기술의 잠재적 위험성에 초점을 맞추고 있습니다. 윤리적 통제를 통해 잘못된 사용이나 사회적 문제를 예방할 수 있다는 점에서 그 주장은 중요성을 지니고 있습니다. 그러나 이러한 주장은 과도한 규제가 혁신을 저해할 수 있다는 점을 간과하고 있습니다.

결론적으로, AI에 대한 윤리적 통제는 기술 발전을 저해하고 개인의 선택을 제한하며, 문제 해결을 기술적 접근으로 전환하는 데 방해가 될 수 있습니다. AI 기술의 혁신과 사회적 혜택을 고려할 때, 윤리적 통제보다는 혁신적인 접근이 더욱 필요하다고 주장합니다. 이는 AI 기술이 더욱 발전하고 사회에 긍정적인 영향을 미칠 수 있는 기반이 될 것입니다.